In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.io import data, wb
import datetime
import os.path
import math

from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

from sklearn import metrics
    
from sklearn.externals import joblib

import talib as ta


In [33]:
def predict(symbol):
    df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
    if (os.path.isfile('data/'+symbol+'-dividends.csv')):
        df_dividends = pd.read_csv('data/'+symbol+'-dividends.csv', index_col='Date', parse_dates=True)
        df_train = df_train.join(df_dividends)
        df_train['Dividends'].fillna(0, inplace=True)
    else:
        df_train['Dividends']=0
        

    pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['Dividends'].rolling(window=10, center=False).mean()
    pma20 = df_train['Close'].rolling(window=20,center=False).mean() + df_train['Dividends'].rolling(window=20, center=False).mean()
    pma50 = df_train['Close'].rolling(window=50,center=False).mean() + df_train['Dividends'].rolling(window=50, center=False).mean()
    pma200 = df_train['Close'].rolling(window=200,center=False).mean() + df_train['Dividends'].rolling(window=200, center=False).mean()
    vma50 = df_train['Volume'].rolling(window=50, center=False).mean()

    #distance from MAs
    diffma10  = (df_train['Close']/pma10 -1)*100
    diffma20 = (df_train['Close']/pma20 -1)*100
    diffma50 = (df_train['Close']/pma50 -1)*100
    diffma200 = (df_train['Close']/pma200 -1)*100

    #tangential of ma lines
    pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
    pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
    pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
    pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

    #sequence of higher high/higher low
    hh = df_train['High'][1:] + df_train['Dividends'][1:] > df_train['High'][:-1] 
    hl = df_train['Low'][1:] + df_train['Dividends'][1:] > df_train['Low'][:-1] 
    hhx1 = hh.shift(1)
    hhx2 = hh.shift(2)
    hlx1 = hl.shift(1)
    hlx2 = hl.shift(2)

    #rolling average of hh and hl
    hh10 = hh.rolling(window=10, center=False).mean()
    hl10 = hl.rolling(window=10, center=False).mean()

    nvx1 = df_train['Volume']/vma50

    rhigh = (df_train['High']/df_train['Close'] - 1)*100
    rlow = (df_train['Low']/df_train['Close'] - 1)*100
    ropen = (df_train['Open'][1:]/df_train['Close'][:-1].values-1)*100
    rbody = ((df_train['Open'] - df_train['Close'])/df_train['Close'])*100

    rhighx1 = rhigh.shift(1)
    rlowx1 = rlow.shift(1)
    ropenx1 = ropen.shift(1)
    rbodyx1 = rbody.shift(1)

    high = df_train['High'].values
    low = df_train['Low'].values
    close = df_train['Close'].values
    #volume = df_train['Volume'].values
    #open = df_train['Open'].values
    #adjclose = df_train['Adj Close'].values

    #rsi = ta.RSI(adjclose, timeperiod=10)/100
    #slowk, slowd = ta.STOCH(high, low, close, 
    #                        fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    #slowk = slowk/100 #feature scaling
    #fastk, fastd = ta.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)

    #macd, macdsignal, macdhist = ta.MACD(close, fastperiod=6, slowperiod=13, signalperiod=4)
    #mfi = ta.MFI(high, low, close, volume/1.0, timeperiod=10)/100
    #bop = ta.SMA(ta.BOP(open, high, low, close),
    #             timeperiod=3)
    
    
    #add features for training data

    #distance from MAs
    df_train = df_train.assign(diffma10=diffma10)
    df_train = df_train.assign(diffma20=diffma20)
    df_train = df_train.assign(diffma50=diffma50)
    df_train = df_train.assign(diffma200=diffma200)

    #tangential of ma lines
    #df_train = df_train.assign(pma10tan=pma10tan)
    #df_train = df_train.assign(pma20tan=pma20tan)
    #df_train = df_train.assign(pma50tan=pma50tan)
    #df_train = df_train.assign(pma200tan=pma200tan)

    df_train = df_train.assign(hh=hh)
    #df_train = df_train.assign(hhx1=hhx1)
    #df_train = df_train.assign(hhx1=hhx2)

    df_train = df_train.assign(hl=hl)
    #df_train = df_train.assign(hlx1=hlx1)
    #df_train = df_train.assign(hlx2=hlx2)

    df_train = df_train.assign(hh10=hh10)
    df_train = df_train.assign(hl10=hl10)

    #last day volume
    df_train = df_train.assign(nvx1=nvx1)

    #candle sticks
    df_train = df_train.assign(ropen=ropen)
    df_train = df_train.assign(rhigh=rhigh)
    df_train = df_train.assign(rlow=rlow)
    df_train = df_train.assign(rbody=rbody)

    df_train = df_train.assign(ropenx1=ropenx1)
    df_train = df_train.assign(rhighx1=rhighx1)
    df_train = df_train.assign(rlowx1=rlowx1)
    df_train = df_train.assign(rbodyx1=rbodyx1)


    #df_train = df_train.assign(rsi=rsi)
    #df_train = df_train.assign(slowk=slowk)
    #df_train = df_train.assign(macd=macd)
    #df_train = df_train.assign(mfi=mfi)
    #df_train = df_train.assign(bop=bop)
    #df_train = df_train.assign(fastk=fastk)
    
    atr = ta.ATR(high, low, close, timeperiod=10)
    df_train.to_csv('data/'+symbol+'-train.csv')
    remove = ['Open','High','Low','Close','Volume','Adj Close','Dividends']
    df_train = df_train.drop(remove, axis=1)
    
    df_train.describe()
    df_train.corr()
    
    X_train = df_train.values[200:-10, :]
    X_test = df_train.values[-10:,:]
    print(len(X_train))

    production = False

    #for hh, shift one position back to get future value for hh
    print("atr=", atr[-1])
    print("predict hh1:(vs actual)")
    y_train = df_train['hh'].shift(-1).values[200:-10].astype(bool)

    model_dir = 'models/predict-hh-'+symbol
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    f_name = model_dir+'/model.pkl'
    if (production):
        model = joblib.load(f_name) 
    else:
        model = train(X_train, y_train)
        joblib.dump(model, f_name) 

    y_test_pred = model.predict_proba(X_test);

    print(df_train['hh'].values[-9:].astype(bool))
    print(y_test_pred[:, 1]>0.5)
    print("prob of higher high ===", y_test_pred[-1,1])

    #for hl
    print("\npredict hl1:(vs actual)")
    y_train = df_train['hl'].shift(-1).values[200:-10].astype(bool)


    model_dir = 'models/predict-hl-'+symbol
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
    f_name = model_dir+'/model.pkl'
    if (production):
        print('loading saved model ...')
        model = joblib.load(f_name) 
    else:
        model = train(X_train, y_train)
        joblib.dump(model, f_name) 


    y_test_pred = model.predict_proba(X_test);

    print(df_train['hl'].values[-9:].astype(bool))
    print(y_test_pred[:, 1]>0.5)
    print("prob of higher low ===", y_test_pred[-1,1])


In [34]:
def train(X_train, y_train):  

    model = RandomForestClassifier(n_estimators=350)     #best so far
    #model = AdaBoostClassifier(n_estimators=100)
    #model = LogisticRegression(random_state=1)
    #model = GradientBoostingClassifier(n_estimators=300)
    #model = KNeighborsClassifier(n_neighbors=15)

    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    print("Precision score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    model.fit(X_train, y_train)
    #print(model.feature_importances_)
    return model



In [35]:
for symbol in ['tsla']:
    print('\npredicting ' + symbol)
    predict(symbol)


predicting tsla
1240
atr= 10.3540450261
predict hh1:(vs actual)
Precision score: 0.68 (+/- 0.02)
[ True False False False  True False  True  True  True]
[ True  True False False  True  True  True False False False]
prob of higher high === 0.211428571429

predict hl1:(vs actual)
Precision score: 0.73 (+/- 0.03)
[ True False False False  True  True  True False  True]
[ True  True False False  True  True  True False  True  True]
prob of higher low === 0.554285714286


In [36]:
def e_predict(X_train, y_train, X_test):

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in [clf1, clf2, clf3, clf4, clf5, eclf]: 
        print(type(clf).__name__, "\n")
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        return y_test_pred
